## Задание 1

В файле находится выборка покупок пользователей. Покупать можно либо трансляцию матча (в title указаны две команды), либо подписку на прямой эфир (все остальное)

id — идентификатор пользователя

created — дата покупки


Новый сезон РПЛ считать начавшимся с 01 июля 2019. Старый сезон — до 1 июля 2019.

Ваша задача развернуть данные в pd.DataFrame

In [14]:
import pandas as pd
import numpy as np
import scipy
import re 
from datetime import datetime, timedelta
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

In [3]:
# Your code should be written below:
data = pd.read_csv('Task 1.csv')
data.head()

,created,title,name,id
0,2018-09-02 19:12:22.144047,Зенит – Спартак. Российская Премьер-лига. 6 тур,Трансляция РПЛ,fa18fdf154aa0defaf386d52f770ff52
1,2018-08-30 13:16:12.620303,Ростов - Рубин. Российская Премьер-лига. 6 тур,Трансляция РПЛ,0ae165a76e8730674f61bb91c0aa108b
2,2018-09-02 19:27:45.044487,Зенит – Спартак. Российская Премьер-лига. 6 тур,Трансляция РПЛ,88b3dc521850617dc465eb7936282357
3,2018-09-01 14:50:46.626348,ЦСКА - Урал. Российская Премьер-лига. 6 тур,Трансляция РПЛ,88dd235c1eeb4be7b39637730e54c121
4,2018-09-01 18:42:08.190768,Краснодар - Локомотив. Российская Премьер-лига...,Трансляция РПЛ,1d6384cc9a6917f2dd1b7df7a9681eb2


## Подсчитайте следующие статистики:
### Q1: Количество уникальных юзеров:

In [4]:
# Your code should be written below:
data['id'].nunique()

163988

Альтернативные способы вывода:

In [5]:
len(data['id'].value_counts())

163988

In [6]:
data.describe()

,created,title,name,id
count,410302,410302,410302,410301
unique,410300,281,29,163988
top,2019-07-27 16:44:41.248637,Прямой эфир канала МАТЧ ПРЕМЬЕР,Трансляция РПЛ,c9252f5f7a89c1ed2a2638faf3a943c4
freq,2,73161,317726,70


#### Количество уникальных пользователей указано во второй строке в последнем столбце: 163 988 шт.

### Q2: Количество уникального контента:

In [7]:
# Your code should be written below
data['title'].nunique()

281

In [8]:
len(data['title'].value_counts())

281

#### Количество уникального контента: 281 шт.

### Q3: Кто наши покупатели:
Кто покупает подписки в новом сезоне — новые пользователи, или у них были и раньше подписки/трансляции? 

In [9]:
# Your code should be written below
# Проверим наличие пропущенных значений и тип данных в столбце created

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 410302 entries, 0 to 410301
Data columns (total 4 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   created  410302 non-null  object
 1   title    410302 non-null  object
 2   name     410302 non-null  object
 3   id       410301 non-null  object
dtypes: object(4)
memory usage: 12.5+ MB


Видим, что одно из значений id пропущено. Так как у нас нет информации о данном пользователе, будем считать его уникальным, заполним его id строкой 'Null'. (Как вариант, можно удалить данную запись, так как она всего одна, то потеря информации будет минимальной).

In [10]:
# Вычислим строку с пропущенным значением
for i in range(data.shape[0]):
    if pd.isnull(data['id'][i])==True:
        print (data.iloc[i])

created         2018-08-04 19:50:42.747908
title      Прямой эфир канала МАТЧ ПРЕМЬЕР
name                          МАТЧ ПРЕМЬЕР
id                                     NaN
Name: 390212, dtype: object


In [11]:
# Заменим пропущенное значение на 'Null'
data.id[390212] = 'Null'

#### Для дальнейшей работы преобразуем столбец created к формату даты.

In [12]:
# Вспомогательная функция для перевода столбца created из строкового типа в формат даты
def convert_to_datetime(row):
    return datetime.strptime(row, '%Y-%m-%d %H:%M:%S.%f')

In [13]:
# Сохраним дату в новый столбец datetime
data['created'] = data['created'].apply(convert_to_datetime)
data.head()

,created,title,name,id
0,2018-09-02 19:12:22.144047,Зенит – Спартак. Российская Премьер-лига. 6 тур,Трансляция РПЛ,fa18fdf154aa0defaf386d52f770ff52
1,2018-08-30 13:16:12.620303,Ростов - Рубин. Российская Премьер-лига. 6 тур,Трансляция РПЛ,0ae165a76e8730674f61bb91c0aa108b
2,2018-09-02 19:27:45.044487,Зенит – Спартак. Российская Премьер-лига. 6 тур,Трансляция РПЛ,88b3dc521850617dc465eb7936282357
3,2018-09-01 14:50:46.626348,ЦСКА - Урал. Российская Премьер-лига. 6 тур,Трансляция РПЛ,88dd235c1eeb4be7b39637730e54c121
4,2018-09-01 18:42:08.190768,Краснодар - Локомотив. Российская Премьер-лига...,Трансляция РПЛ,1d6384cc9a6917f2dd1b7df7a9681eb2


In [36]:
# Зададим дату начала нового сезона
start_date = '2019-07-01 00:00:00.000000'
start_datetime = datetime.strptime(start_date, '%Y-%m-%d %H:%M:%S.%f')

In [44]:
# Посмотрим, есть ли пересечения по пользователям в старом и новом сезонах
old_users = len(set(data[data.created<start_datetime].id) & set(data[data.created>=start_datetime].id))
print ("Кол-во старых пользователей в новом сезоне: {0}".format(old_users))

Кол-во старых пользователей в новом сезоне: 21070


In [45]:
print ("Кол-во новых пользователей в новом сезоне: {0}".format(data[data.created>=start_datetime].id.nunique()-old_users))

Кол-во новых пользователей в новом сезоне: 13535


Проверим себя:

In [46]:
# Общее кол-во уникальных пользователей в новом сезоне
data[data.created>=start_datetime].id.nunique()

34605

In [47]:
13535+21070

34605

#### Видим, что 21 070 пользователей имели в старом сезоне подписку на каналы/трансляции. Соответственно 13 535 пользователей являются новыми.

### Q5: Отдельно рассмотрим тех, у кого в прошлом сезоне было куплено более одной подписки. Хочу прокоммуницировать с этими ребятами.
Что с ними в этом сезоне? Они подписались? Стали покупать разовые? Ничего не купили ещё?

In [48]:
# Your code should be written below
# Проанализируем содержание столбца title
data.title.value_counts()

Прямой эфир канала МАТЧ ПРЕМЬЕР                                              73161
Спартак - ЦСКА. Российская Премьер-лига. 22 тур                              29305
Спартак - Зенит. Российская Премьер-лига. 20 тур                             11809
ЦСКА - Зенит. Российская Премьер-лига. 14 тур                                10211
Ростов - Спартак. Российская Премьер-лига. 23 тур                             9977
                                                                             ...  
Уфа - Анжи. Российская Премьер-Лига. 2 тур                                       2
Урал - Ростов. Российская Премьер-лига. 4 тур                                    2
ИНТЕРШУМ Динамо - Урал. Российская Премьер-лига. 3 тур (Без комментатора)        2
Урал - Динамо Москва. Российская Премьер-Лига. 3 тур                             1
Динамо - Арсенал. 30 тур. МАТЧ ПРЕМЬЕР                                           1
Name: title, Length: 281, dtype: int64

In [49]:
# Создадим список названий всех трансляций матчей
title_broadcast = []
for i in range(data.shape[0]):
    if len(re.findall(r'[а-яА-Я]+\s[-–]\s[а-яА-Я]+', data.title[i]))!=0:
        title_broadcast.append(data.title[i])

In [50]:
# Кол-во купленных трансляций
len(title_broadcast)

337059

In [51]:
data.shape[0]-len(title_broadcast)-73161

82

Видим, что число купленных трансляций равно 337 059 шт. Из таблицы выше видим, что подписок на прямой эфир с названием "Прямой эфир канала МАТЧ ПРЕМЬЕР" было куплено 73 161, остается еще 82 подписки на прямой эфир (82 = 410 302 - 337 059 - 73 161). Заметим, что название данных подписок - 'Российская Премьер-лига. 30 тур. Лучшие моменты всех матчей в прямом эфире':

In [52]:
len(data[data.title=='Российская Премьер-лига. 30 тур. Лучшие моменты всех матчей в прямом эфире'])

82

In [53]:
# Название подписок в столбце title
title_live = ['Прямой эфир канала МАТЧ ПРЕМЬЕР', 'Российская Премьер-лига. 30 тур. Лучшие моменты всех матчей в прямом эфире']

In [54]:
grouped_df = data[(data.created<start_date)&(data.title.isin(title_live))].groupby(['id'])['id'].count()

In [55]:
print('Список id-шников клиентов, которые в старом сезоне купили более одной подписки на прямой эфир')
grouped_df = grouped_df[grouped_df[:]>1].index
grouped_df

Список id-шников клиентов, которые в старом сезоне купили более одной подписки на прямой эфир


Index(['0001da523657aa3186db77e58211c3b1', '000ac2749869663c1060afd11ccf3f55',
       '000cdec294abfc99725c6ac9e4053105', '000cf7e0539886d75f54906d9eaaf3a2',
       '001441a0aa5edade48b624445e73cc84', '0014663637a5fffb696e643afe4cb44a',
       '0018a2370c0cbc338a0c3237b28d2623', '002789d4bdfae818b3c0e237ce884ec5',
       '0027ed35a53a2d60a8bc0eb142f64105', '002b04c10f033cfb603e1696dbaf38b0',
       ...
       'ffd4d0d9430deeb78ea4c4c6c7958d8f', 'ffd7791118a99de30e5ed93b3f1c97a3',
       'ffddfe5c1d1e948f1a48c0ac19ed2b15', 'ffe69139cd15fd9061336944604088d1',
       'ffe7bee35a566669631cd5ee2943d15c', 'ffe8fc02ef0fb1110e9994792eb0a402',
       'ffedef422633aba9901aaa2c0fdfdbbf', 'fff35aa8b259d2224c43d053f0ea9e9a',
       'fffa80e9ae24faf37eca082f28417c52', 'ffffb3ff5862a220b133d089fd1527e9'],
      dtype='object', name='id', length=12276)

In [57]:
old_clients = set(data[data.created>=start_date].id) & set(grouped_df)
print('Общее кол-во уник.кл-в, к-ые в старом сезоне купили >1 подписки и остались в новом сезоне: {0}'.format(len(old_clients)))

Общее кол-во уник.кл-в, к-ые в старом сезоне купили >1 подписки и остались в новом сезоне: 4748


In [59]:
print('Список id-шников клиентов, к-е в старом сезоне купили более одной подписки и в новом сезоне приобрели подписку:') 
list1 = list(set(data[(data.created>=start_date) & (data.title.isin(title_live))].id) & set(grouped_df))
list1[:5]

Список id-шников клиентов, к-е в старом сезоне купили более одной подписки и в новом сезоне приобрели подписку:


['910adb761c1304f66df8afb4e239d83d',
 '392bacadb8aae0a96e2eaa425c5c0c72',
 'ce62371b2ccd9e0e4201e259b2a3f169',
 'a9cfc7ee5cea0cbdd628063500ab92ca',
 '37d80d9ab22514eca04f7ef27e802400']

In [60]:
d1 = len(set(data[(data.created>=start_date) & (data.title.isin(title_live))].id) & set(grouped_df))
print("Кол-во клиентов, к-е в старом сезоне купили более одной подписки и в новом сезоне приобрели подписку: {0}".format(d1))

Кол-во клиентов, к-е в старом сезоне купили более одной подписки и в новом сезоне приобрели подписку: 4026


In [61]:
print('Список id-шников клиентов, к-е в старом сезоне купили более одной подписки, а в новом приобрели разовые трансляции')
list2 = list(set(data[(data.created>=start_date) & (data.title.isin(title_broadcast))].id) & set(grouped_df))
list2[:5]

Список id-шников клиентов, к-е в старом сезоне купили более одной подписки, а в новом приобрели разовые трансляции


['07584bd5664af72c6e0cbf0a08994205',
 '33a948f6a1314eb5c8a79ee21a3873d1',
 '0c20c60ae0f05923e51a3292ef86f224',
 'ffedef422633aba9901aaa2c0fdfdbbf',
 'd86e66b8e96f7518082df69c32406b5c']

In [63]:
d2 = len(set(data[(data.created>=start_date) & (data.title.isin(title_broadcast))].id) & set(grouped_df))
print("Кол-во клиентов, к-е в старом сезоне купили более одной подписки,а в новом приобрели разовые трансляции: {0}".format(d2))

Кол-во клиентов, к-е в старом сезоне купили более одной подписки,а в новом приобрели разовые трансляции: 869


In [64]:
d1+d2

4895

**Пояснение:** сумма кол-ва клиентов, оформивших и не оформивших подписку в новом сезоне, не совпадает с общим кол-вом уникальных клиентов, оставшихся в новом сезоне, так как среди клиентов, оставшихся в новом сезоне, есть одни и те же клиенты, которые и оформили подписку и купили разовую трансляцию. 

In [65]:
print('Список id-шников клиентов, к-е в старом сезоне купили более одной подписки, а в новом еще ничего не купили')
list3 = list(set(grouped_df) - set(data[data.created>=start_date].id))
list3[:5]

Список id-шников клиентов, к-е в старом сезоне купили более одной подписки, а в новом еще ничего не купили


['e69458e51e4c9eaef6d06616365dcd62',
 '6a73784c9a4618873f7f0ab4d9db9e25',
 'b7dd1503f70c82c7cae49bff08a77a57',
 'f97d83c571f42c3244dd58da08ff7dec',
 'd25c7f30daef1b74fcf1f4c220f830dd']

In [66]:
print("Кол-во клиентов, к-е в старом сезоне купили более одной подписки,а в новом еще ничего не купили: {0}".format(len(list3)))

Кол-во клиентов, к-е в старом сезоне купили более одной подписки,а в новом еще ничего не купили: 7528


In [67]:
# Проверим себя
7528 + 4748

12276

### Q6: Может, уже есть те, кто покупал трансляции 2 и более раз? Им выгоднее подписка. Думаем сделать под них специальный оффер и прокоммуницировать. Сколько таких?

In [68]:
# Your code should be written below
grouped_df_2 = data[(data.created>=start_date)&(data.title.isin(title_broadcast))].groupby(['id'])['id'].count()

In [69]:
print('Список id-ков клиентов, к-ые в новом сезоне купили более 2-х трансляций')
grouped_df_2 = grouped_df_2[grouped_df_2[:]>=2].index
grouped_df_2

Список id-ков клиентов, к-ые в новом сезоне купили более 2-х трансляций


Index(['000ac53f8ce5bc67c054c1d064a1a5c3', '000b7c45e21bb416ea8cf32ede7fe6dc',
       '0014665298d1b60835c86bddad531fa5', '0016541260a279a7bc31f370eb0d12f0',
       '0016f197b6f32134929b6f5685dcdaf7', '002a4880a2e553198e7349e806ea6705',
       '00365ecbea3950b9165fc7c7e2e26790', '003b3e2141fee09099b037b8b2a2f940',
       '004a5f85ffa77001431eaf65f8c6d1c7', '00694bdab709e1762f541b24a15ac1a7',
       ...
       'ff9a89199bb72315de868f2eb48d3619', 'ffae4f7be8243eaeedce2214527cb9c3',
       'ffb3f5e34319fdbfb542416a1fae36f3', 'ffc0acbf62efa0ccdc9a45fe157c1788',
       'ffc48600bc6e1927e99228b10e1b4ef3', 'ffca8aba18e1ac5f5e4fa66a966b8235',
       'ffdbe3ffd2d87749fd572dbee5594263', 'ffede257249f916e508f39e774d780fd',
       'ffedef422633aba9901aaa2c0fdfdbbf', 'fff85e2e621f578705cd5469e3985a8a'],
      dtype='object', name='id', length=4919)

In [70]:
print('Кол-во клиентов, к-ые в новом сезоне уже купили более двух трансляций: {0}'.format(len(grouped_df_2)))

Кол-во клиентов, к-ые в новом сезоне уже купили более двух трансляций: 4919


#### Особый фокус можно сделать на клиентах, которые и в прошлом сезоне купили более одной подписки и в новом уже купили более двух трансляций. Посмотрим, сколько таких клиентов.

In [71]:
print('Кол-во клиентов, к-ые в старом сезоне купили более одной подписки и в новом сезоне уже купили более двух трансляций: {0}'
      .format(len(set(grouped_df) & set(grouped_df_2))))

Кол-во клиентов, к-ые в старом сезоне купили более одной подписки и в новом сезоне уже купили более двух трансляций: 227


### Q7: А матчи с какими командами покупали больше всего в старом и новом сезоне? Какая команда лидирует по покупкам?

In [75]:
# Your code should be written below

# Создадим список всех матчей в разрезе по сезонам
matchs_old = []
matchs_new = [] 
for i in range(data.shape[0]):
    if data.created[i]<start_datetime:   
        if len(re.findall(r'[а-яА-Я]+\s[-–]\s[а-яА-Я]+', data.title[i]))!=0:
            matchs_old.append(re.findall(r'[а-яА-Я]+\s[-–]\s[а-яА-Я]+', data.title[i]))
    else: 
        if len(re.findall(r'[а-яА-Я]+\s[-–]\s[а-яА-Я]+', data.title[i]))!=0:
            matchs_new.append(re.findall(r'[а-яА-Я]+\s[-–]\s[а-яА-Я]+', data.title[i]))

In [76]:
df_old = pd.DataFrame(matchs_old, columns=['matchs'])
df_old.matchs.value_counts()[:3]

Спартак - ЦСКА     29305
Спартак - Зенит    11809
ЦСКА - Зенит       10211
Name: matchs, dtype: int64

#### В старом сезоне чаще всего покупали матчи между Спартак - ЦСКА

In [77]:
df_new = pd.DataFrame(matchs_new, columns=['matchs'])
df_new.matchs.value_counts()[:3]

Ростов - Спартак    6914
Тамбов - Спартак    3626
Сочи - Зенит        3042
Name: matchs, dtype: int64

#### В новом сезоне чаще всего покупали матчи между Ростов - Спартак

In [78]:
# Создадим список всех матчей без разбиения на сезоны
matchs = []
for i in range(data.shape[0]):   
    if len(re.findall(r'[а-яА-Я]+\s[-–]\s[а-яА-Я]+', data.title[i]))!=0:
            matchs.append(re.findall(r'[а-яА-Я]+\s[-–]\s[а-яА-Я]+', data.title[i]))

In [79]:
df = pd.DataFrame(matchs, columns=['matchs'])
df.matchs.value_counts()[:3]

Спартак - ЦСКА      29305
Ростов - Спартак    16863
Спартак - Зенит     11809
Name: matchs, dtype: int64

#### Без разбиения на сезоны по количеству покупок лидирует Спартак - ЦСКА

Выше мы посмотрели какие **матчи** лидируют по продажам. Теперь посмотрим, какие **команды** в лидерах:

In [118]:
# Создадим список всех команд в старом сезоне:
commands_old = []
for i in range(df_old.shape[0]):
    coms = re.split(r'\s[-–]\s', df_old.matchs[i])
    commands_old.append(coms[0])
    commands_old.append(coms[1])

In [119]:
from collections import Counter
Counter(commands_old).most_common()[:3]

[('Спартак', 145063), ('Зенит', 88364), ('ЦСКА', 82853)]

#### В старом сезоне лидиром продаж среди команд является Спартак

In [120]:
# Создадим список всех команд в новом сезоне:
commands_new = []
for i in range(df_new.shape[0]):
    coms = re.split(r'\s[-–]\s', df_new.matchs[i])
    commands_new.append(coms[0])
    commands_new.append(coms[1])

In [121]:
Counter(commands_new).most_common()[:3]

[('Спартак', 10969), ('Ростов', 7783), ('Зенит', 7640)]

#### В новом сезоне лидиром продаж среди команд также является Спартак

## Задание 2

Вы провели А/Б-тест двух посадочных страниц. На вариант А зашли 6 352 человека, 462 из них сконвертировались. На вариант Б зашли 7 123 человека, это дало 556 конверсий. Можно ли по результатам теста определить, какой вариант лучше?

### Для оценки будем использовать Z-критерий для двух долей

In [1081]:
group1=[1 for i in range(462)]+[0 for i in range(5890)]
group2=[1 for i in range(556)]+[0 for i in range(6567)]

#### Для начала построим доверительные интервалы для доли конверсий в А и В вариантах:

### Интервальные оценки долей

$$\frac1{ 1 + \frac{z^2}{n} } \left( \hat{p} + \frac{z^2}{2n} \pm z \sqrt{ \frac{ \hat{p}\left(1-\hat{p}\right)}{n} + \frac{z^2}{4n^2} } \right), \;\; z \equiv z_{1-\frac{\alpha}{2}}$$ 

In [1082]:
conf_interval_group1 = proportion_confint(sum(group1), len(group1), method = 'wilson')
conf_interval_group2 = proportion_confint(sum(group2), len(group2), method = 'wilson')

In [1083]:
print('95%% доверительный интервал для доли конверсии, вариант А: [%f, %f]' % conf_interval_group1)
print('95%% доверительный интервал для доли конверсии, вариант В: [%f, %f]' % conf_interval_group2)

95% доверительный интервал для доли конверсии, вариант А: [0.066601, 0.079381]
95% доверительный интервал для доли конверсии, вариант В: [0.072052, 0.084517]


#### Видим, что интервалы пересекаются, соответственно на основе 95% доверительных интервалов нельзя сделать однозначный вывод о том, какой вариант лучше.

### Z-критерий для разности долей (независимые выборки)

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$

In [1]:
# Функция для построения доверительного интервала для разности двух долей
def proportions_diff_confint_ind(sample1, sample2, alpha = 0.05):    
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    
    p1 = float(sum(sample1)) / len(sample1)
    p2 = float(sum(sample2)) / len(sample2)
    
    left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/ len(sample1) + p2 * (1 - p2)/ len(sample2))
    
    return (left_boundary, right_boundary)

In [1085]:
# Функция для расчета z-статистики
def proportions_diff_z_stat_ind(sample1, sample2):
    n1 = len(sample1)
    n2 = len(sample2)
    
    p1 = float(sum(sample1)) / n1
    p2 = float(sum(sample2)) / n2 
    P = float(p1*n1 + p2*n2) / (n1 + n2)
    
    return (p1 - p2) / np.sqrt(P * (1 - P) * (1. / n1 + 1. / n2))

In [1086]:
# Функция для расчета p-value
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [1087]:
print("95%% доверительный интервал для разности двух долей: [%f, %f]" %\
      proportions_diff_confint_ind(group1, group2))

95% доверительный интервал для разности двух долей: [-0.014246, 0.003598]


Доверительный интервал содержит 0, соответсвенно опять не можем сделать вывод о том, какой вариант лучше.

In [1088]:
print("p-value: %f" % proportions_diff_z_test(proportions_diff_z_stat_ind(group1, group2), 'two-sided'))

p-value: 0.243061


#### На уровни значимости 0.05 мы не можем отвергнуть гипотезу о том, что доли конверсий в обоих вариантах одинаковы.

## Задание 3
Если вам надо быстро сравнить много продуктов из схожей категории и вы можете выбрать только две метрики для сравнения, какие вы возьмете?

### Две основные продуктовые метрики - это Retention и LTV.

**Retention** – ключевая метрика, характеризующая то, насколько полезным находят продукт пользователи. Если часть новых пользователей начинает пользоваться продуктом регулярно (кривая Retention выходит на плато), значит, для определенной части рынка продукт приносит пользу, то есть достигнут product/market fit.

**Lifetime value** – метрика, которая показывает, сколько денег средний новый пользователь принесет за все время использования продукта. Эта метрика сочетает в себе все другие метрики монетизации продукта. 

LTV = (Доля платящих от новых) * (Прибыль с платящего пользователя за все время)

Также дополнительно можно отметить такую метрику как **Конверсия во вторую покупку**: нам критически важно понимать, как складывается жизнь клиентов после того, как они начинают пользоваться нашим продуктом. Для этого хорошо подходят конверсионные метрики. Под **Конверсией во вторую покупку**, как правило, понимают долю тех покупателей, которые, сделав первую покупку в течение периода N, сделали вторую в течение либо периода "N", либо периода "N+1". Периоды можно гибко выбирать под конкретную задачу. Конверсия во вторую покупку в разрезе по продуктам говорит нам о том, насколько ценным оказался данный продукт для пользователя и насколько качественно мы работаем с клиентами после продажи. 